In [1]:
!pip install datasets

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout # Changed LSTM to GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from datasets import load_dataset
import pandas as pd
import re
import string
import numpy as np


c:\Users\naman\anaconda3\envs\ml-dl-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# --- Text Preprocessing Functions (Same as before) ---
# Load the dataset
try:
    dataset = load_dataset('csv', data_files='WELFake_Dataset.csv')
except FileNotFoundError:
    print("WELFake_Dataset.csv not found. Attempting to load 'davanstrien/WELFake' from Hugging Face Hub.")
    dataset = load_dataset("davanstrien/WELFake")


WELFake_Dataset.csv not found. Attempting to load 'davanstrien/WELFake' from Hugging Face Hub.


In [4]:

if 'train' in dataset:
    data_split = dataset['train']
else:
    data_split = dataset


In [5]:

def combine_text(example):
    combined_text = ""
    if 'title' in example and example['title'] is not None:
        combined_text += str(example['title'])
    if 'text' in example and example['text'] is not None:
        if combined_text:
            combined_text += " "
        combined_text += str(example['text'])
    
    if not combined_text.strip():
        return {"full_text": ""}
    return {"full_text": combined_text}


In [6]:

data_split = data_split.map(combine_text)
data_split = data_split.remove_columns(['title', 'text']) if 'title' in data_split.column_names and 'text' in data_split.column_names else data_split


Map: 100%|██████████| 72134/72134 [00:08<00:00, 8625.11 examples/s]


In [7]:

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) 
    text = text.strip()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [8]:

data_split = data_split.map(lambda example: {"full_text": clean_text(example["full_text"])})


Map: 100%|██████████| 72134/72134 [00:16<00:00, 4299.60 examples/s]


In [9]:

if 'label' in data_split.column_names:
    data_split = data_split.rename_column("label", "labels")

print("Sample preprocessed text:", data_split['full_text'][0])


Sample preprocessed text: law enforcement on high alert following threats against cops and whites on 911by blacklivesmatter and fyf911 terrorists video no comment is expected from barack obama members of the fyf911 or fukyoflag and blacklivesmatter movements called for the lynching and hanging of white people and cops they encouraged others on a radio show tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in americaone of the fyoflag organizers is called sunshine she has a radio blog show hosted from texas called sunshine s fing opinion radio show a snapshot of her fyf911 lolatwhitefear twitter page at 953 pm shows that she was urging supporters to call now fyf911 tonight we continue to dismantle the illusion of white below is a snapshot twitter radio call invite fyf911the radio show aired at 1000 pm eastern standard timeduring the show callers clearly call for lynching and killing of white peoplea 239 minute clip from the

In [10]:

# --- Dataset Splitting ---
train_test_split = data_split.train_test_split(test_size=0.2, stratify_by_column="labels", seed=42)

train_texts = [ex['full_text'] for ex in train_test_split['train']]
train_labels = np.array([ex['labels'] for ex in train_test_split['train']], dtype=np.int32)

test_texts = [ex['full_text'] for ex in train_test_split['test']]
test_labels = np.array([ex['labels'] for ex in train_test_split['test']], dtype=np.int32)


In [11]:

# --- Keras TextVectorization and Padding ---
MAX_TOKENS = 10000
MAX_SEQUENCE_LENGTH = 256


In [12]:

print("\nBuilding vocabulary with TextVectorization...")
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH,
    standardize=None 
)



Building vocabulary with TextVectorization...


In [13]:

vectorize_layer.adapt(train_texts)

VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
print(f"Vocabulary size: {VOCAB_SIZE}")
print("Sample vocabulary:", vectorize_layer.get_vocabulary()[:20])

train_sequences = vectorize_layer(tf.constant(train_texts)).numpy()
test_sequences = vectorize_layer(tf.constant(test_texts)).numpy()

print("\nTrain Sequences Example (first 5 values):", train_sequences[0][:5])
print("Train Labels Example:", train_labels[0])


Vocabulary size: 10000
Sample vocabulary: ['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'in', 'that', 'is', 'for', 'on', 'it', 'he', 'with', 'was', 'as', 'said', 'by', 'trump']

Train Sequences Example (first 5 values): [3145 3674    1  124  902]
Train Labels Example: 1


In [14]:

# --- GRU Model Definition (TensorFlow Keras) ---
EMBEDDING_DIM = 100
GRU_UNITS = 256 # Units for the GRU layer
DROPOUT_RATE = 0.5
NUM_CLASSES = 2

model = Sequential([
    tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,)),
    Embedding(input_dim=VOCAB_SIZE, 
              output_dim=EMBEDDING_DIM,
              input_length=MAX_SEQUENCE_LENGTH,
              mask_zero=True),
    
    # Changed from LSTM to GRU
    Bidirectional(GRU(GRU_UNITS, return_sequences=False)), 
    
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])


c:\Users\naman\anaconda3\envs\ml-dl-env\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:

# --- Compile the Model ---
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 256, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 512)            │       549,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,550,914 (5.92 MB)

 Trainable params: 1,550,914 (5.92 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:

# --- Training Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_gru_tf_model.keras', save_best_only=True, monitor='val_loss', mode='min')


In [17]:

# --- Train the Model ---
EPOCHS = 10
BATCH_SIZE = 64

print(f"\nStarting training for {EPOCHS} epochs with batch size {BATCH_SIZE}...")
history = model.fit(
    train_sequences,
    train_labels,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(test_sequences, test_labels),
    callbacks=[early_stopping, model_checkpoint]
)
print("\n--- Training Finished ---")



Starting training for 10 epochs with batch size 64...
Epoch 1/10
902/902 ━━━━━━━━━━━━━━━━━━━━ 981s 1s/step - accuracy: 0.8833 - loss: 0.2485 - val_accuracy: 0.9779 - val_loss: 0.0621
Epoch 2/10
902/902 ━━━━━━━━━━━━━━━━━━━━ 975s 1s/step - accuracy: 0.9867 - loss: 0.0406 - val_accuracy: 0.9769 - val_loss: 0.0648
Epoch 3/10
902/902 ━━━━━━━━━━━━━━━━━━━━ 1910s 2s/step - accuracy: 0.9944 - loss: 0.0172 - val_accuracy: 0.9749 - val_loss: 0.0873
Epoch 4/10
902/902 ━━━━━━━━━━━━━━━━━━━━ 861s 955ms/step - accuracy: 0.9976 - loss: 0.0084 - val_accuracy: 0.9791 - val_loss: 0.0700

--- Training Finished ---


In [18]:

# --- Evaluate the Model on the Test Set ---
print("\n--- Evaluating Model on Test Set ---")
loss, accuracy = model.evaluate(test_sequences, test_labels, batch_size=BATCH_SIZE)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


--- Evaluating Model on Test Set ---
226/226 ━━━━━━━━━━━━━━━━━━━━ 68s 300ms/step - accuracy: 0.9781 - loss: 0.0625
Test Loss: 0.0621
Test Accuracy: 97.79%


In [23]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# Predictions
y_pred_probs = model.predict(test_sequences, batch_size=BATCH_SIZE)

# If softmax output (N, 2) → take column for positive class
if y_pred_probs.shape[1] == 2:
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_pred_probs_pos = y_pred_probs[:, 1]  # probability of class 1
else:
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    y_pred_probs_pos = y_pred_probs.flatten()

# If labels are one-hot → convert to 1D
if len(test_labels.shape) > 1 and test_labels.shape[1] > 1:
    test_labels = np.argmax(test_labels, axis=1)
else:
    test_labels = test_labels.flatten()

# Classification Report
print("\n--- Classification Report ---")
print(classification_report(test_labels, y_pred))

# Confusion Matrix
print("\n--- Confusion Matrix ---")
print(confusion_matrix(test_labels, y_pred))

# ROC-AUC
roc_auc = roc_auc_score(test_labels, y_pred_probs_pos)
print(f"\nROC-AUC Score: {roc_auc:.4f}")


226/226 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      7006
           1       0.97      0.99      0.98      7421

    accuracy                           0.98     14427
   macro avg       0.98      0.98      0.98     14427
weighted avg       0.98      0.98      0.98     14427


--- Confusion Matrix ---
[[6762  244]
 [  75 7346]]

ROC-AUC Score: 0.9978
